Documentacion de TimeSeries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Esta función es para atender los clicks cuando se grafiaca algo

MEJORAS: 
   
   1)Si se tocan dos puntos agarra el primero arbitrariamente,estaría bueno un mini menu (Altium) para elegir cual
   
   2)La zona donde los clics son vistos como en la señal es un circulo de radio d al rededor del punto de la señal, el posible problema es que la escala del eje Y que no es siempre la misma haga que este valor no tenga mucho sentido, por lo que debería variar segun el valor que toma
             

In [ ]:
def picker_handler( line, mouseevent):

    if mouseevent.xdata is None:        
        return False, dict()
    
    xdata = line.get_xdata()           
    ydata = line.get_ydata()    
    maxd = 0.3
    d = abs(xdata - mouseevent.xdata)

    ind, = np.nonzero(d <= maxd)
    if len(ind):
        pickx = xdata[ind[0]]
        picky = ydata[ind[0]]
        props = dict(ind=ind, pickx=pickx, picky=picky)
        return True, props
    else:
        return False, dict()

A partir de acá defino la clase
Dentro del dataFrame además de la serie de datos y los tiempos también hay datos para hacer mejores las gráficas y debería haber tambien una columna de vectores donde se guarden los indices de las filas "padre"

MEJORAS:

1) Más funciones estadisticas y de filtrado

2) Hacer bien el indexado

3) Chequear mejor el tema de que no me pasen cualquier cosa en el constructor

In [ ]:
class timeSerie:

    def __init__(self,  typeParam   = None, 
                        origin      = None,           
                        param       = None,      t_param = None, 
                        param_aux1  = None,      t_param_aux1 = None, 
                        param_aux2  = None,      t_param_aux2 = None,
                        abstraLevel = 1,
                        origIndex   = None
                 ):
        
        self.type       = typeParam
        self.__origin__ = origin
        self.__level__  = abstraLevel 
        
        if typeParam == 'Angle':
            self.units    = "º"
        elif typeParam == 'IntervalRR':
            self.units    = "s"            
        elif typeParam == 'SlopeMax':
            self.units    = "mm/mm"        
        
        if type(param) == pd.core.frame.DataFrame:
            self.dataF  = param
        else:
            self.dataF  = pd.DataFrame({   'Time'      : t_param,
                                             typeParam  : param
                                             })   
            if typeParam == 'Angle':
                self.dataF['SlopeMax']      = param_aux1
                self.dataF['TimeSlopeMax']  = t_param_aux1   
                self.dataF['SlopeMin']      = param_aux2
                self.dataF['TimeSlopeMin']  = t_param_aux2   
                
            elif typeParam == 'SlopeMax':
                self.dataF['TimeSlopeMax']  = t_param_aux1  

Grafica la señal mediante la cual se pueden inspeccionar los elementos

MEJORAS:

1) Màs lindo

2) Botones

In [ ]:
    def plotSerie(self):
        fig = plt.figure( 'TimeSerie: ' + self.type + ' plot' + ' (' + str(self.__level__) + ')')
        plt.cla()
        plt.plot( self.dataF['Time'],  self.dataF[ self.type ], 'go', picker=picker_handler)
        fig.canvas.mpl_connect('pick_event',  self.__plotInspection__)
        plt.xlabel('time (s)')   
        plt.ylabel(self.units)
        plt.grid()

Plotea la señal vs la señal que le dio origen, desde aca no se puede inspeccionar

In [ ]:
 def plotSerieVsOrig( self ): 
        if isinstance(self.__origin__, np.ndarray):
            fig = plt.figure( 'TimeSerie: ' + self.type + ' vs Origin')
            plt.cla()
            
            ax1 = plt.subplot(211)
            plt.plot( self.__origin__[:,0],  self.__origin__[:,1])
            plt.setp(ax1.get_xticklabels(), fontsize=6)
            plt.grid()
            
            ax2 = plt.subplot(212, sharex=ax1)
            plt.plot( self.dataF['Time'],  self.dataF[ self.type ], 'go', picker=picker_handler)
            plt.setp(ax2.get_xticklabels(), fontsize=6)
            fig.canvas.mpl_connect('pick_event',  self.__plotInspection__)
            
            plt.grid()
            
        elif isinstance(self.__origin__, timeSerie): 
            fig = plt.figure( 'TimeSerie: ' + self.type + ' vs Origin')
            plt.cla()
            
            ax1 = plt.subplot(211)
            plt.plot( self.__origin__.dataF['Time'],  self.__origin__.dataF[self.__origin__.type] )
            plt.setp(ax1.get_xticklabels(), fontsize=6)
            plt.grid()
            
            ax2 = plt.subplot(212, sharex=ax1)
            plt.plot( self.dataF['Time'],  self.dataF[ self.type ], 'go', picker=picker_handler)
            plt.setp(ax2.get_xticklabels(), fontsize=6)
            fig.canvas.mpl_connect('pick_event',  self.__plotInspection__)
            plt.grid()
                      
        else:
            print('Origen no conocido')
            self.plotSerie()
     

Función que se llama cuando se realiza un clic acertado, dependiendo de que instancia de abstracción es que ploteo realiza

In [ ]:
    def __plotInspection__(self,event):

        if self.__level__ == 1:
            if   self.type == 'IntervalRR':  
                 self.__plotItemRR__( self.__origin__, event.pickx, event.picky )
                
            elif self.type == 'SlopeMax':
                 self.__plotItemSlope__( self.__origin__, event.pickx, event.picky )
                
            elif self.type == 'Angle':
                 self.__plotItemAngle__( self.__origin__, event.pickx, event.picky )
        else:
            self.__origin__.plotSerie()

Funciones para dibujar un complejo QRS

In [ ]:
ef __plotItemRR__ (self,originECG, dt, dy):
        """
        Esto debería ver que es mejor si poner el funciones aca adentro o por fuera
        """
        referencia  = min( originECG[ originECG[:,0] == dt, 1 ], originECG[ originECG[:,0] == dt-dy, 1 ] )[0]                            
        puntos      = np.array([ [dt,referencia] , [dt-dy,referencia] ])           
                
        #ZoomRegion
        timeZR = originECG[ np.bitwise_and( originECG[:,0] >= dt-1.2,  originECG[:,0] <= dt+0.6), 0 ] 
        ecgZR  = originECG[ np.bitwise_and( originECG[:,0] >= dt-1.2,  originECG[:,0] <= dt+0.6), 1 ]
        
        #Ploteo
        plt.figure('Inspection: Intervalo RR')
        plt.cla()
        plt.plot( timeZR,       ecgZR                                                                   )
        plt.plot( puntos[:,0],  puntos[:,1],            label=('Ocurrencia: '+str(np.round(dt,4))+'s')  )
        plt.plot( puntos[:,0],  puntos[:,1],    'ro',   label=('Ocurrencia: '+str(np.round(dt,4))+'s')  )
        plt.xlabel('time (s)')  
        plt.ylabel('amplitude (mV)')      
        plt.grid()
        plt.legend()
        plt.show()
        
        
    def __plotItemSlope__ (self, originECG, dt, dy ):

        #Punto de pendiente maxima 
        punto      = ( dt, originECG[ originECG[:,0] == dt, 1 ] )                                                            
            
        #Recta con valor de pendiente maxima que pasa por el punto 
        len_rec     = 0.01
        t_rectas    = np.array( [ dt - len_rec, dt + len_rec ] )        
        rectas      = np.array( [ punto[1] - dy/0.4*len_rec, punto[1] + dy/0.4*len_rec] )
        
        #ZoomRegion
        timeZR = originECG[ np.bitwise_and( originECG[:,0] >= dt-0.4,  originECG[:,0] <= dt+0.6), 0 ] #zoom_region/self.fs
        ecgZR  = originECG[ np.bitwise_and( originECG[:,0] >= dt-0.4,  originECG[:,0] <= dt+0.6), 1 ] #self.__origin__.signal[zoom_region] 
        
        #Ploteo
        plt.figure('Inspection: Pendiente Máxima')
        plt.cla()
        plt.plot( timeZR,       ecgZR                                                                   )
        plt.plot( t_rectas,     rectas,                 label=('Pendiente: ' +str(np.round(dy,4))    )  )
        plt.plot( punto[0],     punto[1],    'ro',      label=('Ocurrencia: '+str(np.round(dt,4))+'s')  )
        plt.xlabel('time (s)')   
        plt.ylabel('amplitude (mV)')           
        plt.grid()
        plt.legend()
        plt.show()  
        
        
    def __plotItemAngle__ (self, originECG, dt, dy ):
    
#        i = DE ALGUNA FORMA ENCONTRAR EL i CON DT QUE NO SE REPITE NUNCA, ver alguna forma mejor
#        Ojo que filas no es lo mismo que indice
        i = self.dataF[self.dataF['Time']==dt].index
        i = i[0]
        aux = self.dataF.loc[i]
        dt1 = aux[:]['TimeSlopeMax']
        dt2 = aux[:]['TimeSlopeMin']
        m1  = aux[:]['SlopeMax']
        m2  = aux[:]['SlopeMin'] 


        #Puntos de pendiente maxima 
        punto1      = ( dt1, originECG[ originECG[:,0] == dt1, 1 ] )    
        punto2      = ( dt2, originECG[ originECG[:,0] == dt1, 1 ] )  
        
        #Recta con valor de pendiente maxima que pasa por el punto 
        len_rec     = 0.005
        
        t_rectas1    = np.array( [ dt1 - len_rec,                           dt  + len_rec                       ] )        
        rectas1      = np.array( [ punto1[1] - m1/0.4*len_rec,              punto1[1] + m1/0.4*(dt-dt1+len_rec) ] )
        t_rectas2    = np.array( [ dt  - len_rec,                           dt2 + len_rec                       ] )        
        rectas2      = np.array( [ punto2[1] - m2/0.4*(dt2-dt+len_rec),     punto2[1] + m2/0.4*len_rec          ] )
      
        #ZoomRegion
        timeZR = originECG[ np.bitwise_and( originECG[:,0] >= dt1-0.4,  originECG[:,0] <= dt2+0.4), 0 ] 
        ecgZR  = originECG[ np.bitwise_and( originECG[:,0] >= dt1-0.4,  originECG[:,0] <= dt2+0.4), 1 ]
        
        #Ploteo
        plt.figure('Inspection: Pendiente Máxima y Minima que forman el angulo')
        plt.cla()
        plt.plot( timeZR,       ecgZR                                                                   )
        plt.plot( t_rectas1,         rectas1,                 label=('Pendiente Positiva: '   +str(np.round(m1,4))+'('+str(np.round(dt1,4))+'s)')   )
        plt.plot( t_rectas2,         rectas2,                 label=('Pendiente Negativa: '   +str(np.round(m2,4))+'('+str(np.round(dt2,4))+'s)')   )
        plt.plot( punto1[0],        punto1[1],    'ro',       label=('Ocurrencia: '           +str(np.round(dt,4))+'s'  )   )
        plt.plot( punto2[0],        punto2[1],    'ro',       label=('Angulo: '               +str(np.round(dy,4))+'º')   )
        plt.xlabel('time (s)')      
        plt.ylabel('amplitude (mV)')  
        plt.grid()
        plt.legend()
        plt.show()              

Funciones para recortar en el eje X o Y la TimeSerie, no genera otra instancia sino que recorta la actual

In [ ]:
    def recorteX(self, lInf, lSup):
        if lInf == lSup:
            print('Los limites deben ser diferentes')
        else:
            self.dataF = self.dataF.loc[ (self.dataF['Time']>lInf) & (self.dataF['Time']<lSup) ]
        
    def recorteY(self, lInf, lSup):
        if lInf == lSup:
            print('Los limites deben ser diferentes')
        else:
            self.dataF = self.dataF.loc[ (self.dataF[self.type]>lInf) & (self.dataF[self.type]<lSup) ]